In [1]:
import numpy as np
import matplotlib.pyplot as plt 
import matplotlib.ticker as ticker
import heapq
import os
from decimal import Decimal
np.set_printoptions(threshold=np.nan) 
home_path = os.getcwd().replace('BC1/notebooks','')
import cPickle as pickle
import pandas as pd
from scipy.signal import find_peaks
import json
import copy
import matplotlib as mpl



In [2]:
with open("separate_BC.json", "r") as read_file:
    json_file = json.load(read_file)
name_json = json_file["file_name"]
split = name_json.split('.')
run_number = split[1][2:7]

In [3]:

single_anode_channel = int(json_file["single_anode_channel"])
number_tubes_json = len(json_file["tubes"])
input_name_json = json_file["file_name"]

In [4]:
def find_tubes(signal, percentage, number_tubes):
    """
    Function to find the first and last indexes of each
    tube inside the cell. In other words, this function
    get the signal when the cesium capsule is inside
    the cell.
    Parameters:
    ----------
    signal : array
        Array with signal profile from Cesium Scan.
    percentage : float
        Constant number of the percentage to cut the
        ADC Counts axis of the signal, in order to
        find the tubes in the cell.
    number_tubes : int
        Expected number of tubes inside the cell.
    Returns:
    -------
    indexes : list
        List with tuples containing the first and
        last indexes of each tube found.
    """
    # get peak indexes
    height = np.max(signal) * percentage
    peak_idx, _ = find_peaks(signal, height=height)

    # distance between peaks
    peak_dist = np.diff(peak_idx)

    # positional argument
    pos = np.array([i+1 for i, dist in enumerate(peak_dist) if dist >= 100])
    pos = np.insert(pos, 0, 0)
    pos = np.insert(pos, len(pos), len(peak_dist)+1)

    # loop througout all the tubes
    indexes = []
    for tube in range(number_tubes):
        # get the peaks indexes for each tube
        peaks_per_tube = peak_idx[pos[tube]:pos[tube+1]]

        # get mean distance between peaks/periods
        mean_period = np.mean(np.diff(peaks_per_tube))

        # get first and last indexes of this tube
        first = int(peaks_per_tube[0] - (6*mean_period))
        last = int(peaks_per_tube[-1] + (6*mean_period))
        indexes.append((first, last))

    return indexes


In [5]:
#opening the file
dictionary_path = home_path+'processed_data/Dados BC1/'
file = open(dictionary_path+input_name_json+'.pkl','rb')
cesium_scan = pickle.load(file)

single_data = cesium_scan['data'][:, single_anode_channel]

#Path to the channel reading mapping file
map_path = home_path+'processed_data/Dados BC1/mapping.txt'

In [6]:
# Find the tubes inside the cell using the
# peaks information of the single anode PMT
tubes_idx = find_tubes(single_data,percentage=0.6,number_tubes=number_tubes_json)

In [7]:
#Phill's function(No change)
def create_tensor(data, map_file):
    """
    Create tensor from signal profile of the MA-PMT.
    In other words, stack snapshots of the MA-PMT image
    for each time interval.
    Parameters:
    ----------
    data : 2D array
        rows - samples/measurements
        columns - channels/features
    map_file : str
        Complete path of the mapping file.
    Returns:
    -------
    data_tensor : 3D array
        Tensorial format of the data.
    """
    n_samples = data.shape[0]
    data_tensor = np.zeros([8, 8, n_samples])

    # get mapping for image pixels
    pixel_map = np.loadtxt(map_file)

    # nested loop for the grid of pixels
    for i in range(8):
        for j in range(8):
            if pixel_map[i, j] == 0:
                data_tensor[i, j, :] = 0
            else:
                channel = int(pixel_map[i, j]-1)
                data_tensor[i, j, :] = data[:, channel]

    return data_tensor


In [8]:
#Phill's function(No change)
def check_scan_direction(file_name, verbose=True):
    """
    Function to check if the cesium scan was made Forward or Backward
    and if the cell is on the positive (A) or negative (C) side of
    the pseudo-rapidity.
    This is done by splitting the string, containing the name of the
    ROOT data, and retrieving information by the scan coded on the
    file name.
    Parameters:
    ----------
    file_name : string
        Name of the cesium scan ROOT file. Encoded as:
            - A1.CS99999F.LBA66
    verbose : boolean (optional)
        Verbose variable to print information in the screen.
        - default - True
    Returns:
    -------
    scan_info : dict
        Dictionary with useful informations about the cesium scan.
    """
    # split the file name and extract informations
    split = file_name.split('.')
    cell = split[0]
    run_number = split[1][2:7]
    direction = split[1][-1]
    barrel = split[2][:2]
    module = split[2][3:]
    side = split[2][2]
    connector = split[3]

    # print information
    if verbose:
        print('\n>> Data Characteristics <<')
        print('  - Run: %s' % run_number)
        print('  - Cell: %s' % cell)
        print('  - Connector: %s' % connector)
        print('  - Barrel: %s' % barrel)
        print('  - Module: %s' % module)
        print('  - Side: %s' % side)
        print('  - Scan direction: %s' % direction)

    # dictionary holding the direction constants
    direction_const = {'F': 1, 'B': -1}

    # auxiliary constants for each tube
    if barrel == 'LB':
        tubes_const = np.asarray([-1, 1, -1, 1, -1, 1, -1, 1, -1, 1, -1])
        # the indexes are in the physical scale
        tubes_idx = {'A': np.asarray([0, 1, 2]),
                     'BC': np.asarray([3, 4, 5, 6, 7, 8]),
                     'D': np.asarray([9, 10])}
    elif barrel == 'EB':
        print('Direction of Extended Barrel (EB) not implemented yet!')
        sys.exit()
    else:
        print('Not a valid name for TileCal barrel.')
        print('Possible names are: LB and EB')
        sys.exit()

    # check cells and barrels
    cell_digit = filter(lambda x: x.isalpha(), cell)
    tube_direction = tubes_const[tubes_idx[cell_digit]] \
    * direction_const[direction]

    if direction == 'F':
        tube_order = np.flip(tubes_idx[cell_digit], 0) + 1
    elif direction == 'B':
        tube_order = tubes_idx[cell_digit] + 1
    else:
        print('Not a valid scan direction.')
        print('Possible values are: F and B')
        sys.exit()

    # add to results
    scan_info = {'tube_order': tube_order,
                 'tube_direction': tube_direction,
                 'n_tubes': len(tubes_idx[cell_digit])}

    return scan_info

In [9]:
# check scan direction and cell side
scan_info = check_scan_direction(input_name_json)


>> Data Characteristics <<
  - Run: 10706
  - Cell: BC1
  - Connector: FOCON
  - Barrel: LB
  - Module: 66
  - Side: A
  - Scan direction: F


In [10]:
def separate_cell(single_anode_channel,scan_info):
    """
    Function to separate the data from the cesium scan,
    separating them into cells B and C.
    
    Parameters:
    ----------
    single_anode_channel : int
    Position of the channel "Single Anode"  
    
    scan_info : function 
         Function results {"scan_info"}
    Returns:
    -------
    separated_cell : dict
        Dictionary with separation between B and C cells from cesium scanning.
    """
    
    separated_cell={}
    results = {}
    index = []
    results['single_anode'] = single_anode_channel 
    check = [0,0,0,3,3,3]
    for tube, idx in enumerate(tubes_idx):

        results.setdefault('data',[]).append(cesium_scan['data'][idx[0]:idx[1], :])
        # getting only a few indexes
        if ((tube - check[tube])==0):
            index.append(idx[0])
        if ((tube - check[tube])==2):
            index.append(idx[1])

            results.setdefault('trigger',[]).append(cesium_scan['trigger'][index[0]:index[1]])
            results.setdefault('timesec',[]).append(cesium_scan['timesec'][index[0]:index[1]])
            results.setdefault('timemksec',[]).append(cesium_scan['timemksec'][index[0]:index[1]])
            index = []


         # check if is needed to inverse the data
        if scan_info['tube_direction'][tube] == -1:
            #count up to 2 twice(tube - check[tube])
            results['data'][tube - check[tube]] = np.flip(results['data'][tube - check[tube]], axis=0)

         # create tensor images       
        results.setdefault('tensor',[]).append(create_tensor(results['data'][tube - check[tube]], map_path))

        if (tube == 2):
            #deep copy will copy all contents by value.
            separated_cell["DataB"] = copy.deepcopy(results)
            #clean dictionary
            results = {}
        if (tube == 5):
            separated_cell["DataC"] = copy.deepcopy(results)
            results = {}
    return separated_cell

In [11]:
results = separate_cell(single_anode_channel,scan_info)

In [12]:
pickle.dump(results,open(dictionary_path+input_name_json+"_separate.pkl", "wb"))
